### Building Multiple Models Using Macros in SAS

To fit more than one models using PROC LOGISTIC using the variables in a dataset, multiple PROC LOGISTIC statements are usually written. Here, two macros are written to run PROC LOGISTIC n times in a loop, where n is the length of a list of predictor variables of interest. The same technique can be applied to PROC REG or other procedures that only accept one MODEL statement within the procedure. 

A partition of a dataset, where the response variable y has binary outcome, is used to demonstrate the macros. Quantitative and qualitative variables are both presented in the dataset. 

In [ ]:
*example dataset, response variable is y; 
data t;
infile datalines  delimiter =',';
input y  x1$ x2 x3$ x4$ x5 x6 x7$;
id = _N_;

datalines;
1,1,18,4,2,1049,21,1
1,1,9,4,0,2799,36,1
1,2,12,2,9,841,23,1
1,1,12,4,0,2122,39,2
1,2,24,2,3,3758,23,1
1,1,11,4,0,3905,36,1
1,1,30,4,1,6187,24,1
1,1,6,4,3,1957,31,1
1,2,48,3,10,7582,31,1
1,1,18,2,3,1936,23,1
1,1,6,2,3,2647,44,1
0,1,11,4,0,3939,40,1
0,2,18,2,3,3213,25,1
1,2,36,4,3,2337,36,1
0,4,11,4,0,7228,39,1
0,1,6,4,0,3676,37,1
;


The syntax and technique for fitting a logistic model with quantitative and qualitative variables is similar. When the qualitative variables are coded by numbers, it's necessary to use a CLASS statement to ensure that the variable is interpreted as characters. 

In the macro, logistic_quan or logistic_qual, a list of n variables of interest is parsed in the macro, its length is obtained using system function countw(). A do loop is run n times, with iteractor ii, which is incremented 1 unit at the end of each loop. The ii-th variable in the list is located using scan(list, index, type), and it's used as the predictor variable in PROC LOGISTIC. 

In [ ]:
*list of quantitative variables;
%let quan_vars = x2 x5 x6;

*list of qualitative variables;
%let qual_vars = x1 x3 x4 x7;

*logistic model with quantitative variables;
%macro logistic_quan();
    %let n_quan= %sysfunc(countw(&quan_vars));
    %do ii=1 %to &n_quan;
        %let var = %scan(&quan_vars, &ii, %str( ));
        proc logistic data=t;
            model y(event='1')= &var; 
        run;
    %end;
%mend logistic_quan;

*logistic model with qualitaive variables;
%macro logistic_qual();
    %let n_qual = %sysfunc(countw(&qual_vars));
    %do ii=1 %to &n_qual;
        %let var = %scan(&qual_vars, &ii, %str( ));
        proc logistic data=t;
            class &var(param=ref ref=first);
            model y(event='1')=&var;
        run;
    %end
%mend logistic_qual;

In [ ]:
*calling logistic_quan;
%logistic_quan;

*calling logistic_qual;
%logistic_qual

The macros above output the default from PROC LOGISTIC. Additional testings can be added in the PROC LOGISTIC portion of the code. 

The data for each model could be saved, and combined into one dataset as shown below. The first proc logistic model is run, and the results from the test of interest (in this case, the global test) is saved in dataset globaltest_all. A column called 'x' that records the name of the predictor variable is added to the dataset in the proceeding DATA step. This similar process is repeated inside the macro: 1)run PROC LOGISTIC, save test result 2)add variable name to 'x' column 3)append dataset to a main dataset.  

In [ ]:

proc logistic data=t1 outest=out;
	class x1(param=ref ref=first);
	model y(event='1')=x1;
	ods output GlobalTests=globaltest_all;
run;

data globaltest_all;
	set globaltest_all;
	length x $ 3;
	x='x1';
run;

%macro logistic_qual_out();
    %let n= %sysfunc(countw(&qual_vars));
	%do ii=1 %to &n;
		%let var = %scan(&qual_vars, &ii, %str( ));
		proc logistic data=t1;
	 		class &var(param=ref ref=first); 
			model y(event='1')=&var;
			ods output GlobalTests=globaltest&ii;
		run;
		data globaltest&ii;
			set globaltest&ii;
			x= "&var";
		run;
		data globaltest_all;
			set globaltest_all globaltest&ii;
		run;
	%end;
%mend logistic_qual_out;